## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sergiyevsk,RU,53.9413,51.1650,38.70,53,100,15.26,overcast clouds
1,1,Pacific Grove,US,36.6177,-121.9166,70.11,70,1,3.00,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,70.12,66,100,4.52,overcast clouds
3,3,Airai,TL,-8.9266,125.4092,62.11,84,91,1.52,overcast clouds
4,4,Horsham,GB,51.0640,-0.3272,59.65,68,40,10.36,scattered clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Puerto Ayora,EC,-0.7393,-90.3518,75.16,87,96,5.99,overcast clouds
11,11,Hilo,US,19.7297,-155.0900,76.62,71,20,4.61,few clouds
14,14,Alofi,NU,-19.0595,-169.9187,75.51,69,69,19.71,broken clouds
15,15,Houma,US,29.5958,-90.7195,84.99,79,69,15.01,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,75.18,89,0,5.73,clear sky
25,25,Caravelas,BR,-17.7125,-39.2481,80.92,60,83,10.36,broken clouds
27,27,Uvira,CD,-3.4067,29.1458,76.71,73,53,3.15,broken clouds
28,28,Diego De Almagro,CL,-26.3667,-70.0500,83.64,12,0,13.62,clear sky
31,31,Pauini,BR,-7.7136,-66.9764,81.09,79,79,1.86,broken clouds
34,34,Kailua,US,21.4022,-157.7394,80.08,73,75,3.44,haze


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## No empty rows

In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
11,Hilo,US,76.62,few clouds,19.7297,-155.0900,
14,Alofi,NU,75.51,broken clouds,-19.0595,-169.9187,
15,Houma,US,84.99,broken clouds,29.5958,-90.7195,
21,Kapaa,US,75.18,clear sky,22.0752,-159.3190,
25,Caravelas,BR,80.92,broken clouds,-17.7125,-39.2481,
27,Uvira,CD,76.71,broken clouds,-3.4067,29.1458,
28,Diego De Almagro,CL,83.64,clear sky,-26.3667,-70.0500,
31,Pauini,BR,81.09,broken clouds,-7.7136,-66.9764,
34,Kailua,US,80.08,haze,21.4022,-157.7394,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Hilo,US,76.62,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Alofi,NU,75.51,broken clouds,-19.0595,-169.9187,Taloa Heights
15,Houma,US,84.99,broken clouds,29.5958,-90.7195,Courtyard by Marriott Houma
21,Kapaa,US,75.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
641,Atambua,ID,75.99,broken clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
646,Tabou,CI,78.53,overcast clouds,4.4230,-7.3528,hotêl doufoulougou
647,Borazjan,IR,76.42,overcast clouds,29.2666,51.2159,شرکت زرمهریار شعبه سوم
648,Kirakira,SB,80.02,broken clouds,-10.4544,161.9205,SanBiz Lodge


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(15,15), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))